In [ ]:
import geopandas as gp
import pandas as pd
import numpy as np
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    SentinelHubRequest,
    bbox_to_dimensions,
    to_utm_bbox,
    wgs84_to_pixel
)
import rasterio

In [13]:
import os
SENTINEL_CLIENT_ID = os.getenv('SENTINEL_CLIENT_ID')
SENTINEL_CLIENT_SECRET = os.environ.get('SENTINEL_CLIENT_SECRET')

In [14]:
from sentinelhub import SHConfig

config = SHConfig()

config.sh_client_id = SENTINEL_CLIENT_ID
config.sh_client_secret = SENTINEL_CLIENT_SECRET

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")
else:
    print("Successfully set up SentinelHub client")


Successfully set up SentinelHub client


In [4]:
county_fields = gp.read_file("./counties_rand_fields.geojson").to_crs("WGS84");

In [ ]:
fields = county_fields.explode()

In [6]:
deg_to_m = 3.3 # ~375km

In [7]:
fields["x"] = np.floor(fields["geometry"].x / deg_to_m) * deg_to_m
fields["y"] = np.floor(fields["geometry"].y / deg_to_m) * deg_to_m

In [8]:
evalscript_NVDI_bands = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B04", "B08"]
            }],
            output: {
                bands: 2
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B08];
    }
"""

In [18]:
resolution = 150
response = None
_150m_to_deg = 0.0015;

def fetch_sentinel(data, year, month):
    year_s = str(year)
    month_s = str(month)
    border = data["geometry"].total_bounds
    border[2] = max(border[2], border[0] + _150m_to_deg);
    border[3] = max(border[3], border[1] + _150m_to_deg);

    border = BBox(tuple(border), crs=CRS.WGS84)
    dims = bbox_to_dimensions(border, resolution=resolution)

    # Indexing SRC: https://forum.sentinel-hub.com/t/get-output-coordinates-using-sentinelhubpy/2804/2
    bb_utm = to_utm_bbox(border)
    transform = bb_utm.get_transform_vector(resx=resolution, resy=resolution)
    entries = []
    # Sentinel data is available from 2017

    request = SentinelHubRequest(
        evalscript=evalscript_NVDI_bands,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(year_s + "-" + month_s + "-01",
                                year_s + "-" + month_s + "-06"),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=border,
        size=dims,
        config=config,
    )

    img_data = request.get_data()[0]

    for d in data.itertuples():
      # Compute the xy coordinate
      x, y = wgs84_to_pixel(d.geometry.x, d.geometry.y,
                            transform, utm_epsg=bb_utm.crs)
      x = min(max(x, len(img_data)), 0)
      y = min(max(y, len(img_data[x])), 0)

      pixel = img_data[x][y]
      B04 = pixel[0]
      B08 = pixel[1]
      NVDI = (B08-B04)/(B08+B04)

      entries.append([year, month, d.NAME, d.STATE_NAME, NVDI])

    return pd.DataFrame(entries, columns=["year", "month", "name", "state_name", "NDVI"])


In [21]:
import concurrent.futures

In [25]:
def fetch_month(yearMonth):
    year, month = yearMonth;
    path = "./" + str(year) + "-" + str(month) + "-ndvi.csv";
    if os.path.isfile(path):
      print(path, "already exists")
      return;
    print("fetching", year, month);
    sentinel_data = fields.groupby(["x","y"]).apply(lambda x: fetch_sentinel(x, year, month));
    # Drop invalid ndvi values
    valid_ndvi = sentinel_data.reset_index(drop = True).dropna()
    valid_ndvi = valid_ndvi.drop(valid_ndvi[((valid_ndvi["NDVI"] > 1) | (valid_ndvi["NDVI"] < 0))].index)
    valid_ndvi.to_csv(path)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
  executor.map(fetch_month, [(year, month) for year in range(2017, 2023) for month in range(1,13)])